# TensorFlow2 Keras 推理


```{topic} 主题
TVM 对 TensorFlow1 模型的支持并不完善，为了提高开发效率，本节讨论如何将 TensorFlow1 模型转换为 TensorFlow2 的 Keras 模型，最终再将其转换为 ONNX 或者 TFLite 模型。
```

参考：[migrating_checkpoints](https://www.tensorflow.org/guide/migrate/migrating_checkpoints)

下面以模型 [resnet_v2_50](http://download.tensorflow.org/models/resnet_v2_50_2017_04_14.tar.gz) 为例展示。

需要克隆项目 [models](https://github.com/tensorflow/models)，然后执行如下操作。

In [1]:
import tensorflow as tf
try:
    tf1 = tf.compat.v1
except (ImportError, AttributeError):
    tf1 = tf
tf.get_logger().setLevel('ERROR')

2023-06-17 20:53:43.044085: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-17 20:53:43.092131: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-17 20:53:43.093054: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-17 20:53:43.960849: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


切换到 `models/research/slim` 目录下：

In [2]:
%cd /media/pc/data/lxw/ai/tasks/models/research/slim

/media/pc/data/lxw/ai/tasks/models/research/slim


将 TF1 升级为 TF2：

In [3]:
from nets import resnet_v2
import tf_slim as slim

class ResnetV2_50(tf.keras.Model):
    def __init__(self, trainable=False, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.trainable = trainable

    @tf.function(input_signature=[tf.TensorSpec([None, 3, 224, 224], 
                                                 tf.float32, name="data")])
    @tf1.keras.utils.track_tf1_style_variables
    def call(self, x):
        x = tf.convert_to_tensor(x, tf.float32) # 确保输入是 tensor
        x = tf.transpose(x, perm=(0, 2, 3, 1)) # NCHW -> NHWC
        with slim.arg_scope(resnet_v2.resnet_arg_scope()):
            logits, end_points = resnet_v2.resnet_v2_50(
                x, 
                num_classes=1001,
                global_pool=True,
                is_training=self.trainable,
                scope="resnet_v2_50"
            )
        del end_points
        return tf.nn.softmax(logits)

预处理：

In [4]:
from PIL import Image
import numpy as np
from nets import resnet_v2
from preprocessing.preprocessing_factory import get_preprocessing
import tf_slim as slim

preprocessing = get_preprocessing("resnet_v2_50")
image_size = 224
path = '/media/pc/data/board/arria10/lxw/data/test/cat.png' # 将要预测的图片路径
preprocessing = get_preprocessing("resnet_v2_50")


@tf.function
def preprocess_image(image, output_height, output_width):
    # image = tf.constant(image)
    processed_image = preprocessing(image, output_height, output_width)
    return processed_image/256
with Image.open(path) as im:
    if im.mode != "RGB":
        im.convert("RGB")
    image = np.asarray(im)
np_processed_image = preprocess_image(image, image_size, image_size)
np_processed_images = np.expand_dims(np_processed_image, axis=0)
np_processed_images = np_processed_images.transpose(0, 3, 1, 2)

2023-06-17 20:53:45.795888: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


前向推理：

In [5]:
model = ResnetV2_50()
model(tf.ones(shape=(1, 3, 224, 224), dtype=tf.float32))
ckpt = tf.train.Checkpoint(model=model)
ckpt_path = "/media/pc/data/board/arria10/lxw/tests/npu_user_demos/models/resnet50_v2_tf/weight/resnet_v2_50.ckpt"
ckpt.restore(ckpt_path) # 更新模型参数
outputs = model(np_processed_images)
outputs = outputs.numpy()

/media/pc/data/tmp/cache/conda/envs/tvmz/lib/python3.10/site-packages/tensorflow/python/keras/engine/base_layer.py:2212: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
/media/pc/data/tmp/cache/conda/envs/tvmz/lib/python3.10/site-packages/tensorflow/python/keras/engine/base_layer.py:1345: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`layer.updates` will be removed in a future version. '
/media/pc/data/tmp/cache/conda/envs/tvmz/lib/python3.10/site-packages/keras/legacy_tf_layers/base.py:627: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  self.updates, tf.compat.v1.GraphKeys.UPDATE_OPS


打印标签信息：

In [6]:
from github import Github

g = Github(user_agent="xinetzone")
repo = g.get_repo("tensorflow/models")
label_content = repo.get_contents("research/slim/datasets/imagenet_lsvrc_2015_synsets.txt")
imagenet_labels = label_content.decoded_content.decode().split()
assert len(imagenet_labels) == 1000
metadata = repo.get_contents("research/slim/datasets/imagenet_metadata.txt")
imagenet_metadata = metadata.decoded_content.decode().splitlines()
synset_to_human = {}
for metadata in imagenet_metadata:
    name, value = metadata.split("\t")
    synset_to_human[name] = value
name2id = {name: k+1 for k, name in enumerate(imagenet_labels)}

topk = 5
sorted_inds = outputs[0].argsort()[::-1]
for sorted_ind in sorted_inds[:topk]:
    label = synset_to_human[imagenet_labels[sorted_ind-1]]
    print(f"{sorted_ind-1}: {label.ljust(20)}\t{outputs[0][sorted_ind-1]}")

282: tiger cat           	0.15822742879390717
285: Egyptian cat        	0.0009194915182888508
281: tabby, tabby cat    	9.054027759702876e-05
278: kit fox, Vulpes macrotis	0.0044543202966451645
277: red fox, Vulpes vulpes	2.409669548342208e-07


In [7]:
model.summary()

Model: "resnet_v2_50"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 25,615,849
Trainable params: 0
Non-trainable params: 25,615,849
_________________________________________________________________


将其模型和参数与加载下来：

In [8]:
# # model = ResnetV2_50()
# inputs = tf.keras.Input(shape=(224, 224, 3), dtype=tf.float32, name="data")
# outputs = model(inputs)
# model2 = tf.keras.Model(inputs=inputs, outputs=outputs, name="resnet_v2_50_model")

# model2.save(module_with_signature_path)

In [9]:
module_with_signature_path = "temp/resnet_v2_50_keras"
model.save(module_with_signature_path)
imported_with_signatures = tf.saved_model.load(module_with_signature_path)
infer = imported_with_signatures.signatures['serving_default']
labeling = infer(tf.constant(np_processed_images))

/media/pc/data/tmp/cache/conda/envs/tvmz/lib/python3.10/site-packages/tensorflow/python/keras/engine/base_layer.py:2212: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
/media/pc/data/tmp/cache/conda/envs/tvmz/lib/python3.10/site-packages/tensorflow/python/keras/engine/base_layer.py:1345: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`layer.updates` will be removed in a future version. '
/media/pc/data/tmp/cache/conda/envs/tvmz/lib/python3.10/site-packages/keras/legacy_tf_layers/base.py:627: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  self.updates, tf.compat.v1.GraphKeys.UPDATE_OPS


## 转换为 ONNX 模型

[Keras 模型转换 ONNX](https://onnxruntime.ai/docs/tutorials/tf-get-started.html)：

In [10]:
import tf2onnx
import onnx

input_signature = [tf.TensorSpec([1, 3, 224, 224], tf.float32, name="data")]
onnx_model, _ = tf2onnx.convert.from_keras(model, input_signature)
onnx.save(onnx_model, "temp/resnet_v2_50.onnx")

2023-06-17 20:54:02.030976: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 2
2023-06-17 20:54:02.031157: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2023-06-17 20:54:02.310043: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2023-06-17 20:54:05.298185: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 2
2023-06-17 20:54:05.298350: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2023-06-17 20:54:05.299015: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GP

构建库：

In [11]:
import set_env
from tvm.relay.frontend import from_onnx

shape_dict = {"data": [1, 3, 224, 224]}

graph_def = onnx.load("temp/resnet_v2_50_tf.onnx")
mod, params = from_onnx(
    graph_def,
    shape_dict,
    freeze_params=True
)

推理：

In [12]:
import tvm
from tvm import relay

with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mod, "llvm", params=params)
    
inputs_dict = {"data": np_processed_images}
mlib_proxy = tvm.contrib.graph_executor.GraphModule(lib["default"](tvm.cpu()))
mlib_proxy.run(**inputs_dict)

One or more operators have not been tuned. Please tune your model for better performance. Use DEBUG logging level to see more details.


验证一致性：

In [13]:
np.testing.assert_allclose(
    labeling['output_1'].numpy(), 
    mlib_proxy.get_output(0).numpy(),
    rtol=1e-07, atol=1e-5
)

## 转换为 TFLite 模型

In [14]:
import tensorflow as tf

# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model(module_with_signature_path)
tflite_model = converter.convert()

# Save the model.
with open('temp/resnet_v2_50.tflite', 'wb') as f:
    f.write(tflite_model)

2023-06-17 20:55:10.138710: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-06-17 20:55:10.138752: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2023-06-17 20:55:10.139157: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: temp/resnet_v2_50_keras
2023-06-17 20:55:10.151565: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-06-17 20:55:10.151585: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: temp/resnet_v2_50_keras
2023-06-17 20:55:10.188512: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:353] MLIR V1 optimization pass is not enabled
2023-06-17 20:55:10.198890: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2023-06-17 20:55:10.658211: I tensorflow/cc/saved_model/loader.cc:215] Running initialization op on SavedModel bundle at path: temp/resne

加载 TFLite 模型：

In [15]:
import tflite


with open('temp/resnet_v2_50.tflite', "rb") as fp:
    tflite_model_buf = fp.read()

tflite_model = tflite.Model.GetRootAsModel(tflite_model_buf, 0)
mod, params = relay.frontend.from_tflite(
    tflite_model, shape_dict=shape_dict, 
    dtype_dict={"data": "float32"}
)
desired_layouts = {
    # 'image.resize2d': ['NCHW'],
    'nn.conv2d': ['NCHW', 'default'],
    'nn.max_pool2d': ['NCHW', 'default'],
    'nn.avg_pool2d': ['NCHW', 'default'],
}
# NHWC 将布局转换为 NCHW 且移除未使用算子
seq = tvm.transform.Sequential([
    relay.transform.RemoveUnusedFunctions(),
    relay.transform.ConvertLayout(desired_layouts)
])
with tvm.transform.PassContext(opt_level=3):
    mod = seq(mod)

验证结果一致性：

In [16]:
with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mod, "llvm", params=params)
    
inputs_dict = {"data": np_processed_images}
mlib_proxy = tvm.contrib.graph_executor.GraphModule(lib["default"](tvm.cpu()))
mlib_proxy.run(**inputs_dict)
np.testing.assert_allclose(
    labeling['output_1'].numpy(), 
    mlib_proxy.get_output(0).numpy(),
    rtol=1e-07, atol=1e-5
)

AssertionError: 
Not equal to tolerance rtol=1e-07, atol=1e-05

Mismatched elements: 44 / 1001 (4.4%)
Max absolute difference: 0.99999833
Max relative difference: 0.99999833
 x: array([[6.913423e-07, 2.183130e-07, 1.979768e-06, ..., 2.010756e-07,
        1.359403e-06, 4.046750e-06]], dtype=float32)
 y: array([[0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

```{warning}
TFLite 转换出现了问题，暂时搁置。
```